# Python 协程
Python 协程是一种轻量级的并发编程模型，它使得在单线程中实现并发变得简单。协程是通过生成器实现的，它可以在执行过程中让出控制权，让其他协程执行，然后再恢复执行。

与多线程相比，协程的优势在于它只使用一个线程，因此避免了多线程中线程切换带来的开销。此外，协程也没有多线程中的竞争条件，因此也避免了死锁的风险。

Python3.5 开始引入了 async 和 await 关键字，使得协程的使用更加方便。 async 定义一个异步函数， await 用于挂起函数的执行直到某个异步操作完成。


现在你知道gather和wait方法的真正区别了吗？

1.gather具有把普通协程函数包装成协程任务的能力，wait没有。wait只能接收包装后的协程任务列表做参数。

2.两者返回值不一样，wait返回的是已完成和未完成任务的列表，而gather直接返回协程任务执行结果。

3.gather返回的任务执行结果是有序的，wait方法获取的结果是无序的。


In [8]:
import asyncio

async def func1(i):
    print(f"协程函数{i}马上开始执行。")
    await asyncio.sleep(2)
    print(f"协程函数{i}执行完毕!")

async def main():
    tasks = []
    for i in range(1, 5):
        # 这里未由协程函数创建协程任务
        tasks.append(func1(i))

    # 注意这里*号。gather自动将函数列表封装成了协程任务。
    await asyncio.gather(*tasks)

if __name__ == '__main__':
    asyncio.run(main())


#--------------------------



import asyncio

async def func1(i):
    print(f"协程函数{i}马上开始执行。")
    await asyncio.sleep(2)
    print(f"协程函数{i}执行完毕!")


async def main():
    tasks = []
    # 创建包含4个协程任务的列表
    for i in range(1, 5):
        tasks.append(asyncio.create_task(func1(i)))

    await asyncio.wait(tasks)

if __name__ == '__main__':
    asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

我们还可以给每个协程任务通过add_done_callback的方法给单个协程任务添加回调函数，如下所示

In [7]:
 #-*- coding:utf-8 -*-
import asyncio

async def func1(i):
    print(f"协程函数{i}马上开始执行。")
    await asyncio.sleep(2)
    return i

# 回调函数
def callback(future):
    print(f"执行结果:{future.result()}")

async def main():
    tasks = []
    for i in range(1, 5):
        task = asyncio.create_task(func1(i))

        # 注意这里，增加回调函数
        task.add_done_callback(callback)
        tasks.append(task)

    await asyncio.wait(tasks)

if __name__ == '__main__':
    asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

# 设置任务超时
很多协程任务都是很耗时的，当你使用wait方法收集协程任务时，可通过timeout选项设置任务切换前单个任务最大等待时间长度，如下所示：

获取任务执行结果，如下所示：
done, pending = await asyncio.wait(tasks, timeout=10)

自省
asyncio.current_task: 返回当前运行的Task实例，如果没有正在运行的任务则返回 None。如果 loop 为 None 则会使用 get_running_loop()获取当前事件循环。

asyncio.all_tasks: 返回事件循环所运行的未完成的Task对象的集合。

原文链接：https://blog.csdn.net/weixin_42134789/article/details/116956109